<span style="font-size: 14pt">MIPT, Applied ML, Autumn 2018</span>

<span style="font-size: 16pt"> HW #5: Approximate RL homework

<span style="color:red; font-size: 14pt;"> Дедлайн 5.12.2018 12:00 </span>

<span style="color:blue; font-size: 12pt">Valentin Malykh </span>,
<span style="color:blue; font-size: 12pt; font-family: 'Verdana'">val@maly.hk</span>


**Оформление дз**: 
- Выполненное задание требуется отправлять через <a href='https://goo.gl/forms/XPSIbwp7wPxB4SsI3'>форму </a>

- Выполненное дз прикрепляйте в формате файла ``<фамилия>_<группа>_task<номер>.ipynb``, например: ``ivanov_594_task4.ipynb`` 

**Вопросы**:
- Вопросы присылайте в канал в телеграмме ``[Fall 2018]ML Seminars``

--------
- **PS1**: Будьте внимательны при заполнении формы, когда отправляете ДЗ 


<h1 align="center">Практическое задание (100%)</h1>
Описание находится по ссылке: https://gist.github.com/madrugado/bbaedcb614755d47345c3c69b1b41169

**Я реализовала алгоритм Proximal-Policy-Optimization. Я руководствовалась вот этим https://junhongxu.github.io/Proximal-Policy-Optimization/. Автор этого репозитория утверждает, что набирает реворд 275. Однако, вглядевшись в код, я заметила, что он считает реворд, как сумму за 2048 итераций, а не за игру. Если считать так же, как он, то моя реализация набирает реворд 550, а средний реворд за игру 260.**

Суммарно я потратила на это задание около недели, при этом почти с утра до ночи, потому что DDPG не учился, а в PPO я выискивала неочевидные баги(


In [166]:
import gym
import os
import sys
import math
import numpy as np
import matplotlib.pyplot as plt
from torch.autograd import Variable
from torch.distributions import Categorical
from itertools import count
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import random

In [137]:
env = gym.make("BipedalWalker-v2")

In [138]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("USE_CUDA: ", use_cuda)
Tensor = FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if use_cuda else torch.ByteTensor

USE_CUDA:  True


In [139]:
from tqdm import tqdm_notebook

In [140]:
 EPS = 0.003
class Approx(nn.Module):
    def __init__(self, state_size, hid_size=64):
        print(hid_size)
        super(Approx, self).__init__()
        self.full1 = nn.Linear(state_size, hid_size)
        #self.full1.weight.data = fanin_init(self.full1.weight.data.size())
        print(self.full1.weight.data.shape)
        self.full2 = nn.Linear(hid_size, 2 * hid_size)
        print(self.full2.weight.data.shape)
        self.full5 = nn.Linear(2 * hid_size, 1)
        
        print(self.full5.weight.data.shape)
        self.full5.weight.data.mul(0.1)

    def forward(self, x):
        #print("S", x.shape, y.shape)
        x = F.tanh(self.full1(x))
        x = F.tanh(self.full2(x))

        x = self.full5(x)
        return x

In [141]:
def log_normal_density(x, mean, log_std, std):
    variance = std.pow(2)
    log_density = -(x - mean).pow(2) / (2 * variance) - 0.5 *\
        np.log(2 * np.pi) - log_std
    log_density = log_density.sum(dim=1, keepdim=True)
    
    return log_density

In [142]:
class Policy(nn.Module):
    def __init__(self, state_size, action_size, hid_size=64):
        super(Policy, self).__init__()
        
        self.act_fc1 = nn.Linear(state_size, 64)
        self.act_fc2 = nn.Linear(64, 128)
        self.mu = nn.Linear(128, action_size)
        self.mu.weight.data.mul_(0.1)
        # torch.log(std)
        print(self.act_fc1.weight.data.shape)
        print(self.act_fc2.weight.data.shape)
        print(self.mu.weight.data.shape)
        self.logstd = nn.Parameter(torch.zeros(action_size))

    def forward(self, x):
        #print(x.shape)
        act = self.act_fc1(x)
        act = F.tanh(act)
        act = self.act_fc2(act)
        act = F.tanh(act)
        mean = self.mu(act)  # N, num_actions
        logstd = self.logstd.expand_as(mean)
        std = torch.exp(logstd)
        action = torch.normal(mean, std)
        
        logprob = log_normal_density(action, mean, std=std, log_std=logstd)
        
        return action, logprob, mean

In [143]:
def evaluate_actions(policy, approx, x, action):
    v = approx.forward(x)
    _, _, mean = policy.forward(x)
    #print("E", policy.logstd)
    logstd = policy.logstd.expand_as(mean)
    #print("U", logstd)
    std = torch.exp(logstd)
    # evaluate
    logprob = log_normal_density(action, mean, log_std=logstd, std=std)
    dist_entropy = 0.5 + 0.5 * math.log(2 * math.pi) + logstd
    #print("E", dist_entropy)
    dist_entropy = dist_entropy.sum(-1).mean()
    #print("W", dist_entropy.shape)
    return v, logprob, dist_entropy

In [144]:
env.observation_space.shape[0], env.action_space.shape[0]

(24, 4)

In [145]:
env.action_space.shape

(4,)

In [146]:
import time
from torch.utils.data.sampler import BatchSampler, SubsetRandomSampler

In [147]:
def calculate_returns(rewards, dones, last_value, gamma=0.99):
    returns = np.zeros(len(rewards) + 1)
    returns[-1] = last_value
   
    for i in reversed(range(len(rewards))):
        returns[i] = gamma * returns[i+1] * (1 - dones[i]) + rewards[i]
    return returns

In [158]:
class Actor_Critic(object):
    def __init__(self, env, 
                    gamma=0.99,
                    lr=2e-4,
                    test_episodes=30,
                    num_iter=7,
                    batch_size=64,
                    coef_entropy=1e-4, 
                    clip_value=0.2):
        super(Actor_Critic, self).__init__()
        self.num_iter = num_iter
        self.coef_entropy=coef_entropy
        self.clip_value=clip_value
        self.env = env
        self.state_dim, self.action_dim, self.action_lim = (env.observation_space.shape[0], env.action_space.shape[0],
                                                                            env.action_space.high[0])
        
        self.actor = Policy(self.state_dim, self.action_dim)
        

        self.critic = Approx(self.state_dim)
        
        self.optimizer = optim.Adam(list(self.actor.parameters()) + list(self.critic.parameters()),lr=lr)
        
        
        if use_cuda:
            self.actor.cuda()
            self.critic.cuda()
            
        
        self.test_episodes = test_episodes
        self.gamma = gamma

        self.train_rewards = []
        self.test_rewards = []
        self.train_steps = []
        self.test_steps = []
        self.losses_w = []
        self.losses_theta = []
        
        
        
        self.batch_size = batch_size

        
        self.cnt=0
        


    def play_one_step(self, s, it=0):
        "One step"

        action, logprob, mean = self.actor.forward(Variable(Tensor(s[np.newaxis])))
        action, logprob = action.data.cpu().numpy()[0], logprob.data.cpu().numpy()[0]
        value = (self.critic.forward(Variable(Tensor(s[np.newaxis])))).data.cpu().numpy()[0]
        self.observations.append(np.array(s))
        
        self.logprobs.append(logprob)
        self.values.append(value[0])
        self.actions.append(action)
        
        s1, r, is_terminal, _ = self.env.step(action)
        self.rewards.append(r)
        self.dones.append(is_terminal)
        
        s1 = np.float32(s1)

        return s1, r, is_terminal
    
    def play_game(self):
        
        '''Играет фиксированное количество итераций. Может быть несколько игр'''
        t = time.clock()

        state = np.float32(self.env.reset())
        
        #self.history.clear()
        MAX_STEP = 2100
        frames = []
        it = 1
        self.observations = []
        self.rewards = []
        self.logprobs = []
        self.dones = []
        self.values = []
        self.actions= []
        game = 0
        rewards = []
        sum_rewards = []
        steps = []
        while True:
            state1, reward, is_terminal = self.play_one_step(state, it)
            rewards += [reward]
            if is_terminal:
                # начало новой игры
                state = env.reset()
                steps.append(game)
                sum_rewards.append(sum(rewards))
                rewards = []
                game = 0
            #rewards.append(reward)
            it +=1
            game += 1
            state = state1

            if it == MAX_STEP:
                break
     
        last_value = 0
        if not is_terminal:
            obs = Tensor(state1[np.newaxis])
            value = self.critic(obs)
            last_value = value.data[0][0]
        returns = calculate_returns(self.rewards, self.dones, last_value)[:-1]
        
        
        ii = np.arange(len(self.actions)).reshape(-1, 1)
        self.observations = np.array(self.observations)
        self.actions = np.array(self.actions)
        self.logprobs = np.array(self.logprobs).reshape(-1, 1)
        self.rewards = np.array(self.rewards).reshape(-1, 1)
        self.values = np.array(self.values).reshape(-1, 1)
        returns = np.array(returns).reshape(-1, 1)
        advantages = returns - self.values
        #в блоге утверждается, что это ускоряет обучение
        advantages = (advantages - advantages.mean()) / advantages.std()
        
        tt = time.clock()
        
        for e in range(self.num_iter):
            # несколько раз семплируем батчи и обучаемся
            sampler = BatchSampler(SubsetRandomSampler(list(range(advantages.shape[0]))), 
                                   batch_size=self.batch_size,
                                   drop_last=False)
            for i, index in enumerate(sampler):
                sampled = [0 for i in range(8)]
                sampled[0] = Tensor(self.observations[index])

                sampled[1] = Tensor(self.actions[index])
                sampled[2] = Tensor(self.logprobs[index])
                sampled[3] = Tensor(returns[index])
                sampled[4] = Tensor(advantages[index])
                sampled[5] = Tensor(ii[index])
                sampled[6] = Tensor(self.values[index])
                sampled[7] = Tensor(self.rewards[index])
                
                self.optimize(sampled)
        tt = time.clock()
        
        return self.rewards, [it]

    
    def optimize(self, mini_batch):
        '''Optimizes loss on mini_batch'''
        i = mini_batch[5]
        
        s = mini_batch[0]
        a = mini_batch[1]
        logprobs = mini_batch[2]
        v = mini_batch[6]
        r = mini_batch[7]
        returns = mini_batch[3]
        advantages = mini_batch[4]

        
        new_value, new_logprob, dist_entropy = evaluate_actions(self.actor, self.critic, s, a)
        sampled_logprobs = logprobs.view(-1, 1)

        ratio = torch.exp(new_logprob - sampled_logprobs)
        
        sampled_advs = advantages#.view(-1, 1)

        surrogate1 = ratio * sampled_advs
        surrogate2 = torch.clamp(ratio, 1 - self.clip_value, 1 + self.clip_value) * sampled_advs

        policy_loss = -torch.min(surrogate1, surrogate2).mean()
        
        sampled_returns = returns
        
        value_loss = F.mse_loss(new_value, sampled_returns)

        loss =  value_loss - self.coef_entropy * dist_entropy + policy_loss
        
        t = time.clock() 
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        tt = time.clock() 
   
        
    

                
    def train(self, num_episodes, save_ep=1000, path='.'):
        
        self.save_ep = save_ep
        self.q = time.clock()
        for it in tqdm_notebook(range(num_episodes)):
            
            rewards, steps = self.play_game()
        
            self.train_rewards += [sum(rewards)]
            self.train_steps += steps
                
            if (it+1) % 30 == 0:
                print("It: ", it+1, "reward: ", sum(rewards), "steps: ", steps)
                print("Mean reward: ", np.mean(np.array(self.train_rewards[-30:])))
                print("Max reward: ", np.max(np.array(self.train_rewards[-30:])))
            if (it) % (self.save_ep) == 0:
                torch.save(self.actor.state_dict(), "actor_{}.pkl".format(it+1))
                torch.save(self.critic.state_dict(), "critic_{}.pkl".format(it+1))

    

In [159]:
num_episodes = 5000

In [78]:

env = gym.make("BipedalWalker-v2")



agent = Actor_Critic(env)

torch.Size([64, 24])
torch.Size([128, 64])
torch.Size([4, 128])
64
torch.Size([64, 24])
torch.Size([128, 64])
torch.Size([1, 128])


In [79]:
agent.train(5000)

It:  30 reward:  [-97.07760611] steps:  [2100]
Mean reward:  -408.05289308050266
Max reward:  -97.07760610838051
It:  60 reward:  [-269.43642273] steps:  [2100]
Mean reward:  -157.1034998519414
Max reward:  -55.763304690922844
It:  90 reward:  [-43.04090008] steps:  [2100]
Mean reward:  -100.66386217270112
Max reward:  -16.65478804527251
It:  120 reward:  [28.08066106] steps:  [2100]
Mean reward:  -26.252632754756608
Max reward:  28.08066106485138
It:  150 reward:  [-17.80448907] steps:  [2100]
Mean reward:  27.83649370629946
Max reward:  109.48918313607584
It:  180 reward:  [-111.21672501] steps:  [2100]
Mean reward:  50.33197050869353
Max reward:  165.6798149249069
It:  210 reward:  [98.70462825] steps:  [2100]
Mean reward:  14.049158603436588
Max reward:  150.39524475822026
It:  240 reward:  [161.53233768] steps:  [2100]
Mean reward:  93.75829457246383
Max reward:  174.48607814324723
It:  270 reward:  [-52.57596218] steps:  [2100]
Mean reward:  113.87364389986854
Max reward:  210.30

**При таком подходе функция игры возвращает суммарный скор не за одну игру, а за некоторое количество итераций. Для того, чтобы посмотреть на скоры за игру, я написала еще 2 функции**

In [163]:
def test(self, num_episodes):
        
        #self.save_ep = save_ep
        #self.q = time.clock()
        self.test_rewards = []
        self.test_steps = []
        back_rew = len(self.test_rewards)
        back_step = len(self.test_steps)
        sum_rewards = []
        for it in tqdm_notebook(range(num_episodes)):
            
            r, rewards, steps = play_test_game(agent)
        
            self.test_rewards += rewards
            self.test_steps += steps
            sum_rewards += [sum(r)]
            if (it+1) % 10 == 0:
                print("It: ", it+1, "reward: ", rewards, "steps: ", steps)
                print("Sum reward: ", np.mean(np.array(sum_rewards[-10:])))
                print("Mean reward: ", np.mean(np.array(self.test_rewards[back_rew:])))
                print("Max reward: ", np.max(np.array(self.test_rewards[back_rew:])))
                print("Mean steps: ", np.mean(np.array(self.test_steps[back_step:])))
                back_rew = len(self.test_rewards)
                back_step = len(self.test_steps)
           

In [164]:
def play_test_game(self):
        t = time.clock()
        #print("START")
        # Reset env
        state = np.float32(self.env.reset())
        #rewards = []
        self.history.clear()
        MAX_STEP = 4000
        frames = []
        it = 1
        self.observations = []
        self.rewards = []
        self.logprobs = []
        self.dones = []
        self.values = []
        self.actions= []
        game = 0
        rewards = []
        sum_rewards = []
        steps = []
        while True:
            state1, reward, is_terminal = self.play_one_step(state, it)
            rewards += [reward]
            if is_terminal:
                state = env.reset()
                steps.append(game)
                sum_rewards.append(sum(rewards))
                rewards = []
                game = 0
            #rewards.append(reward)
            it +=1
            game += 1
            state = state1
            if it == MAX_STEP:
                break
     
        last_value = 0
        if not is_terminal:
            obs = Tensor(state1[np.newaxis])
            value = self.critic(obs)
            last_value = value.data[0][0]
        returns = calculate_returns(self.rewards, self.dones, last_value)[:-1]
        
        
        ii = np.arange(len(self.actions)).reshape(-1, 1)
        self.observations = np.array(self.observations)
        self.actions = np.array(self.actions)
        self.logprobs = np.array(self.logprobs).reshape(-1, 1)
        self.rewards = np.array(self.rewards).reshape(-1, 1)
        self.values = np.array(self.values).reshape(-1, 1)
        returns = np.array(returns).reshape(-1, 1)
        advantages = returns - self.values
        advantages = (advantages - advantages.mean()) / advantages.std()
        return self.rewards, sum_rewards, steps

In [112]:
test(agent, 35)

It:  10 reward:  [280.24743125883805, 283.6427875553818, 282.2083802269286] steps:  [1086, 1054, 1076]
Sum reward:  993.4522671567717
Mean reward:  258.8415203106822
Max reward:  286.4212890789678
Mean steps:  1047.6451612903227
It:  20 reward:  [282.0364129842332, 280.03263427407256, 276.88403394669996] steps:  [1079, 1064, 1150]
Sum reward:  976.0654892819854
Mean reward:  249.5416343969717
Max reward:  285.2001960127187
Mean steps:  1015.2424242424242
It:  30 reward:  [284.0317188853409, 282.71173157895174, 280.87356460438076] steps:  [1062, 1070, 1085]
Sum reward:  881.542826200887
Mean reward:  195.68406687171347
Max reward:  287.37003667373455
Mean steps:  906.4736842105264


**Реворд 200-250!**

Уменьшим learning rate

In [113]:
agent.optimizer = optim.Adam(list(agent.actor.parameters()) + list(agent.critic.parameters()),lr=2e-5)

In [114]:
agent.train(1000)

/usr/local/lib/python3.5/dist-packages/torch/nn/functional.py:995: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


It:  30 reward:  [418.43166938] steps:  [2100]
Mean reward:  496.9649341639776
Max reward:  569.0230465972048
It:  60 reward:  [565.47033536] steps:  [2100]
Mean reward:  517.6634445615043
Max reward:  566.8098324862757
It:  90 reward:  [560.78574121] steps:  [2100]
Mean reward:  473.8682746008208
Max reward:  571.0199972195215
It:  120 reward:  [437.35090979] steps:  [2100]
Mean reward:  468.7064923502734
Max reward:  577.6538907133813
It:  150 reward:  [533.89286545] steps:  [2100]
Mean reward:  471.43487704697634
Max reward:  562.1608689342512
It:  180 reward:  [525.09702021] steps:  [2100]
Mean reward:  497.92370666680347
Max reward:  565.3669200398966
It:  210 reward:  [389.57678034] steps:  [2100]
Mean reward:  495.0662304411102
Max reward:  563.5162014722368
It:  240 reward:  [553.53618538] steps:  [2100]
Mean reward:  479.74919853275264
Max reward:  559.5932717955366
It:  270 reward:  [517.01128038] steps:  [2100]
Mean reward:  493.68667861510863
Max reward:  561.7754743471135


In [115]:
test(agent, 35)

It:  10 reward:  [278.9206992612655, 283.5884674485009, 283.20438695916107] steps:  [1129, 1046, 1067]
Sum reward:  1002.6315391094515
Mean reward:  271.74705385988403
Max reward:  285.0492349628513
Mean steps:  1079.032258064516
It:  20 reward:  [277.3641827324988, 280.15853992058754, 283.6247429031536] steps:  [1116, 1110, 1054]
Sum reward:  957.2050453199217
Mean reward:  239.72411984310577
Max reward:  287.3153421497251
Mean steps:  1001.3529411764706
It:  30 reward:  [278.7384583420781, 280.79691445006756, 277.0257924492659] steps:  [1119, 1110, 1135]
Sum reward:  987.2706169073577
Mean reward:  257.967324861938
Max reward:  286.5430724761506
Mean steps:  1044.59375


In [118]:
agent.optimizer = optim.Adam(list(agent.actor.parameters()) + list(agent.critic.parameters()),lr=5e-6)

In [119]:
agent.train(1000)

It:  30 reward:  [509.07191881] steps:  [2100]
Mean reward:  517.5569877105959
Max reward:  581.825526426599
It:  60 reward:  [555.15767937] steps:  [2100]
Mean reward:  515.611089918066
Max reward:  565.1600328751657
It:  90 reward:  [534.49638539] steps:  [2100]
Mean reward:  513.8005300940164
Max reward:  565.265537898598
It:  120 reward:  [519.3126806] steps:  [2100]
Mean reward:  497.69290181218423
Max reward:  567.4048598562802
It:  150 reward:  [398.57690148] steps:  [2100]
Mean reward:  500.30087831769936
Max reward:  558.4811678410692
It:  180 reward:  [543.17867061] steps:  [2100]
Mean reward:  504.84762204598496
Max reward:  545.82724101588
It:  210 reward:  [534.28082296] steps:  [2100]
Mean reward:  518.8704995490992
Max reward:  569.2184552037107
It:  240 reward:  [562.91195246] steps:  [2100]
Mean reward:  534.4313828546403
Max reward:  574.4136485943036
It:  270 reward:  [545.10783088] steps:  [2100]
Mean reward:  503.5543800625479
Max reward:  576.6369043364156
It:  30

**Протестируем еще раз**

In [120]:
test(agent, 35)

It:  10 reward:  [285.79285718311496, 285.18671332217014, 285.4787015959271] steps:  [1027, 1056, 1034]
Sum reward:  1033.4723740450122
Mean reward:  257.4712938894956
Max reward:  289.2843609913308
Mean steps:  992.0909090909091
It:  20 reward:  [289.8678382258148, 282.3187649111206, 286.910789411992] steps:  [990, 1087, 1008]
Sum reward:  1042.9667751003867
Mean reward:  257.79210945258916
Max reward:  289.8678382258148
Mean steps:  990.1515151515151
It:  30 reward:  [284.94322414127174, 281.9810024821265, 284.6772572047411] steps:  [1055, 1070, 1046]
Sum reward:  1008.0233299204768
Mean reward:  249.93985852174004
Max reward:  287.88962546116505
Mean steps:  1005.71875


Эксперименты

In [121]:
agent.optimizer = optim.Adam(list(agent.actor.parameters()) + list(agent.critic.parameters()),lr=2e-4)

In [122]:
agent.train(1000)

It:  30 reward:  [552.59441632] steps:  [2100]
Mean reward:  532.3078290818028
Max reward:  594.7081834487027
It:  60 reward:  [546.11368707] steps:  [2100]
Mean reward:  544.90209027139
Max reward:  576.9634278992082
It:  90 reward:  [377.51744177] steps:  [2100]
Mean reward:  509.3816553291921
Max reward:  585.6864958076033
It:  120 reward:  [540.80764884] steps:  [2100]
Mean reward:  484.5868598258657
Max reward:  557.3288273031029
It:  150 reward:  [563.76811649] steps:  [2100]
Mean reward:  478.73615674178996
Max reward:  564.8467233030611
It:  180 reward:  [435.46128448] steps:  [2100]
Mean reward:  477.81649679097956
Max reward:  566.6335103314221
It:  210 reward:  [433.10261831] steps:  [2100]
Mean reward:  459.5372931087723
Max reward:  572.3002658459267
It:  240 reward:  [577.33707628] steps:  [2100]
Mean reward:  476.0205333775392
Max reward:  577.3370762791329
It:  270 reward:  [565.45842088] steps:  [2100]
Mean reward:  471.70477731514274
Max reward:  576.9845703906695
It:

KeyboardInterrupt: 

Стало хуже, поэтому уменьшим лернинг рейт

In [125]:
agent.optimizer = optim.Adam(list(agent.actor.parameters()) + list(agent.critic.parameters()),lr=2e-5)

In [126]:
agent.train(1000)

It:  30 reward:  [535.46574832] steps:  [2100]
Mean reward:  489.8792912712109
Max reward:  558.465690526012
It:  60 reward:  [524.08481884] steps:  [2100]
Mean reward:  518.0632873046168
Max reward:  556.9133857627019
It:  90 reward:  [556.15249662] steps:  [2100]
Mean reward:  497.24779627550754
Max reward:  557.7082806419148
It:  120 reward:  [541.82182775] steps:  [2100]
Mean reward:  509.5005906029242
Max reward:  562.2901667428414
It:  150 reward:  [562.98126631] steps:  [2100]
Mean reward:  530.1070566398846
Max reward:  563.9382865355836
It:  180 reward:  [417.23800667] steps:  [2100]
Mean reward:  518.1811151509647
Max reward:  570.7204511737541
It:  210 reward:  [447.38279544] steps:  [2100]
Mean reward:  518.2208152996234
Max reward:  578.0220678141256
It:  240 reward:  [520.78280343] steps:  [2100]
Mean reward:  531.7928385217821
Max reward:  567.7914074252327
It:  270 reward:  [397.23816501] steps:  [2100]
Mean reward:  534.5511190049486
Max reward:  568.9552389963771
It: 

KeyboardInterrupt: 

In [127]:
test(agent, 30)

It:  10 reward:  [283.3183324934526, 278.26261527390415, 279.8077729158369] steps:  [992, 1063, 1034]
Sum reward:  1008.4706502636029
Mean reward:  244.35730670643525
Max reward:  285.1455442614303
Mean steps:  969.3783783783783
It:  20 reward:  [282.527140613124, 280.7914866575695, 281.043911025913] steps:  [986, 1032, 1014]
Sum reward:  1028.8829056937884
Mean reward:  248.62425345260013
Max reward:  283.72031183452384
Mean steps:  968.8857142857142
It:  30 reward:  [281.04918283496596, 279.29338082414023, 95.09223264834955, 277.32329236835403] steps:  [1019, 1028, 787, 1048]
Sum reward:  1045.3750058935073
Mean reward:  259.24898116955103
Max reward:  283.4929008212321
Mean steps:  992.030303030303


**Итоговый тестовый реворд примерно 250**